# Creating a Custom Robotics Task: Pick Pen Tutorial

In this tutorial, you will learn how to create a complete robotics task from scratch. We'll build a "Pick Pen" task where a robot needs to pick up a mechanical pencil and place it on a plate in a kitchen environment.

## Overview

By the end of this tutorial, you will have created:

- Custom observation functions for detecting task completion
- Custom termination conditions
- Complete environment configuration
- Environment registration for training and data generation

## Task Description

The Pick Pen task involves three main subtasks:

1. **Pick the pen**: Robot approaches and grasps the mechanical pencil
2. **Place on plate**: Robot moves the pen to the plate and releases it
3. **Return to rest**: Robot returns to its rest position

## Step 1: Creating Observation Functions

First, we'll create functions to observe task progress. These functions help us determine when each subtask is completed.

### Exercise 1.1: Complete the Pen Grasping Detection

Navigate to `~/workspace/leisaac/source/leisaac/leisaac/tasks/pick_pen/tasks/pick_pen/mdp/observations.py` and complete the `pen_grasped` function:

```python
def pen_grasped(
    env: ManagerBasedRLEnv,
    robot_cfg: SceneEntityCfg = SceneEntityCfg("robot"),
    ee_frame_cfg: SceneEntityCfg = SceneEntityCfg("ee_frame"),
    object_cfg: SceneEntityCfg = SceneEntityCfg("MechanicalPencil"),
    diff_threshold: float = 0.01,
    grasp_threshold: float = 0.40,
) -> torch.Tensor:
    """Check if an object(Pen) is grasped by the specified robot."""
```

**Your Task**: Complete the logic to determine if the pen is grasped by:

1. Calculate the distance between the end effector and the pen using `torch.linalg.vector_norm()` with `object_pos - end_effector_pos` and set `dim=1`
2. Check if pen is grasped by combining two conditions using `torch.logical_and()`:
   - Distance is less than `diff_threshold`
   - Gripper is closed: `robot.data.joint_pos[:, -1] < grasp_threshold`

### Exercise 1.2: Complete the Pen Placement Detection

Still in `observations.py`, complete the `put_pen_to_plate` function:

```python
def put_pen_to_plate(
    env: ManagerBasedRLEnv,
    robot_cfg: SceneEntityCfg = SceneEntityCfg("robot"),
    ee_frame_cfg: SceneEntityCfg = SceneEntityCfg("ee_frame"),
    object_cfg: SceneEntityCfg = SceneEntityCfg("MechanicalPencil"),
    plate_cfg: SceneEntityCfg = SceneEntityCfg("Plate"),
    x_range: tuple[float, float] = (-0.10, 0.10),
    y_range: tuple[float, float] = (-0.10, 0.10),
    diff_threshold: float = 0.05,
    grasp_threshold: float = 0.60,
) -> torch.Tensor:
    """Check if an object(pen) is placed on the specified plate."""
```

**Your Task**: Complete the logic to determine if the pen is placed on the plate by:

1. Check if pen is within plate X boundaries using `torch.logical_and()` with:
   - `pen_x < plate_x + x_range[1]`
   - `pen_x > plate_x + x_range[0]`
2. Check if pen is within plate Y boundaries using `torch.logical_and()` with:
   - `pen_y < plate_y + y_range[1]`
   - `pen_y > plate_y + y_range[0]`
3. Check if gripper is open: `robot.data.joint_pos[:, -1] > grasp_threshold`
4. Combine pen placement condition with end effector proximity using `torch.logical_and(pen_in_plate, ee_near_to_pen)`
5. Finally combine with gripper open condition using `torch.logical_and(placed, gripper_open)`

## Step 2: Creating Termination Conditions

Termination functions determine when an episode should end, either due to success or failure.

### Exercise 2.1: Complete the Task Completion Check

Navigate to `~/workspace/leisaac/source/leisaac/leisaac/tasks/pick_pen/tasks/pick_pen/mdp/terminations.py` and complete the `task_done` function:

```python
def task_done(
    env: ManagerBasedRLEnv,
    pens_cfg: List[SceneEntityCfg],
    plate_cfg: SceneEntityCfg,
    x_range: tuple[float, float] = (-0.10, 0.10),
    y_range: tuple[float, float] = (-0.10, 0.10),
    height_range: tuple[float, float] = (-0.07, 0.07),
) -> torch.Tensor:
    """Determine if the pen picking task is complete."""
```

**Your Task**: Complete the logic that checks if:

1. Check if pen is within X-range relative to plate using `torch.logical_and()` with:
   - `pen_x < plate_x + x_range[1]`
   - `pen_x > plate_x + x_range[0]`
2. Check if pen is within Y-range relative to plate using `torch.logical_and()` with:
   - `pen_y < plate_y + y_range[1]`
   - `pen_y > plate_y + y_range[0]`
3. Check if pen is within height range relative to plate using `torch.logical_and()` with:
   - `pen_height < plate_height + height_range[1]`
   - `pen_height > plate_height + height_range[0]`
4. Check if robot is at rest pose using `torch.logical_and()` with:
   - `is_so101_at_rest_pose(joint_pos, joint_names)`

## Step 3: Environment Configuration

Now we'll set up the complete environment configuration that brings everything together.

### Exercise 3.1: Complete the Subtask Observation Configuration

Navigate to `~/workspace/leisaac/source/leisaac/leisaac/tasks/pick_pen/tasks/pick_pen/pick_pen_env_cfg.py` and find the `SubtaskCfg` class within `ObservationsCfg`:

```python
@configclass
class SubtaskCfg(ObsGroup):
    """Observations for subtask group."""

    # TODO: Add pen grasped observation term
    pick_pen = None  # YOUR CODE HERE

    # TODO: Add pen placement observation term
    put_pen_to_plate = None  # YOUR CODE HERE
```

**Your Task**: Complete the subtask observation terms by:

1. Create `pick_pen` observation using `ObsTerm()` with:
   - `func=mdp.pen_grasped` (the function you implemented in Step 1)
   - `params={"object_cfg": SceneEntityCfg("MechanicalPencil")}`
2. Create `put_pen_to_plate` observation using `ObsTerm()` with:
   - `func=mdp.put_pen_to_plate` (the function you implemented in Step 1)
   - `params={"object_cfg": SceneEntityCfg("MechanicalPencil"), "plate_cfg": SceneEntityCfg("Plate")}`

### Exercise 3.2: Complete the Termination Configuration

In the same file, find the `TerminationsCfg` class:

```python
@configclass
class TerminationsCfg:
    """Configuration for the termination"""

    time_out = DoneTerm(func=mdp.time_out, time_out=True)

    # TODO: Add success termination condition using task_done function
    success = None  # YOUR CODE HERE
```

**Your Task**: Configure the success termination condition by:

1. Create `success` termination using `DoneTerm()` with:
   - `func=mdp.task_done` (the function you implemented in Step 2)
   - `params={"pens_cfg": [SceneEntityCfg("MechanicalPencil")], "plate_cfg": SceneEntityCfg("Plate")}`

## Step 4: Mimic Environment for Data Generation

The mimic environment is used for generating training data by recording human demonstrations. This environment extends your base environment with additional capabilities for collecting high-quality training data through automated subtask decomposition and trajectory recording.

Navigate to `~/workspace/leisaac/source/leisaac/leisaac/tasks/pick_pen/tasks/pick_pen/pick_pen_mimic_env_cfg.py` and examine the subtask configurations. This file contains the complete configuration for data generation - no modifications are needed as it demonstrates how subtasks are automatically broken down and recorded for training.

The mimic environment automatically:

- Decomposes complex tasks into subtasks (pick_pen → put_pen_to_plate → rest_robot)
- Records demonstration trajectories with proper segmentation
- Applies domain randomization and noise for robust data collection
- Manages task transitions and interpolation between subtask segments

## Step 5: Environment Registration

Finally, we need to register our environments so they can be used for training and data collection.

### Exercise 5.1: Complete Environment Registration

Navigate to `~/workspace/leisaac/source/leisaac/leisaac/tasks/pick_pen/tasks/pick_pen/__init__.py`:

```python
import gymnasium as gym

gym.register(
    # TODO: Set the environment ID for the standard PickPen environment
    id=None,  # YOUR CODE HERE
    entry_point="isaaclab.envs:ManagerBasedRLEnv",
    disable_env_checker=True,
    kwargs={
        "env_cfg_entry_point": f"{__name__}.pick_pen_env_cfg:PickPenEnvCfg",
    },
)

gym.register(
    # TODO: Set the environment ID for the mimic environment
    id=None,  # YOUR CODE HERE
    entry_point=f"leisaac.enhance.envs:ManagerBasedRLLeIsaacMimicEnv",
    disable_env_checker=True,
    kwargs={
        "env_cfg_entry_point": f"{__name__}.pick_pen_mimic_env_cfg:PickPenMimicEnvCfg",
    },
)
```

**Your Task**: Complete the gym environment registration by:

1. Set the standard environment ID to: `"LeIsaac-SO101-PickPen-v0"`
2. Set the mimic environment ID to: `"LeIsaac-SO101-PickPen-Mimic-v0"`

These environment IDs follow the gymnasium naming convention and will be used throughout the training pipeline to identify your custom task.

## Step 6: Creating Scene Configuration

The final step is to create the scene configuration that defines our kitchen environment with the pen object. This configuration tells the system where to find the USD scene file and how to spawn it.

### Exercise 6.1: Complete the Scene Configuration

Navigate to `workspace/leisaac/source/leisaac/leisaac/assets/scenes/kitchen.py` and add the kitchen with pen scene configuration:

```python
# TODO: Add the USD path for the kitchen with pen scene
KITCHEN_WITH_PEN_USD_PATH = None  # YOUR CODE HERE

# TODO: Create the asset configuration for the kitchen with pen scene
KITCHEN_WITH_PEN_CFG = None  # YOUR CODE HERE
```

Congratulations! You've successfully created a complete robotics task from scratch with all necessary components including observation functions, termination conditions, environment configurations, and scene setup.
